# Configuracion

Estos primeros bloques son de importar librerías, y configurar las credenciales de la API para poder descargar

In [ ]:
# Codificación Rodrigo Pereira
# rodrigo.pereira.r@mail.pucv.cl
# Laboratorio de Robótica y Visión, EIE PUCV

In [1]:
# Importar librerías
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image  
import PIL  
from sentinelhub import (SHConfig, DataCollection, SentinelHubCatalog, SentinelHubRequest, BBox, bbox_to_dimensions, CRS, MimeType, Geometry)

/home/noobird/anaconda3/envs/fondecyt/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from sentinelhub import (
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
)

In [3]:
config = SHConfig()

# Ingresar client id
config.sh_client_id = ""
# Ingresar client secret 
config.sh_client_secret = ""
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"
config.save("cdse")

config = SHConfig("cdse")

# Area

Estos bloques son para asignar el área geográfica para descargar. Son Bounding Boxs rectangulares con las coordenadas WGS84

El formato es: Coordenadas Esquina Inferior Izquierda, Coordenadas Esquina Superior Derecha

Se elige también la resolución en metros/pixel y se muestra el tamaño (resolución en pixeles) aproximado de la imagen a descargar. 

> LA API NO ADMITE TAMAÑOS DE IMÁGENES MAYORES A 2500 PIXELES DE ALTO O ANCHO

Las coordenadas están hechas de tal forma que ninguna imagen queda mayor a 2500 pixles por ningún lado.

In [4]:
# Specify the path to your Excel file and the name of the sheet
excel_file = 'dbCoordenadas.xlsx'
sheet_name = 'Sheet1'

# Read the specific column from the Excel file into a DataFrame
column_name = 'B'  # Replace 'A' with the column letter or name you want to read
df = pd.read_excel(excel_file, sheet_name=sheet_name, usecols=column_name)

df['Bbox'] = df['Bbox'].str.slice(start=7)
df['Bbox'] = df['Bbox'].str[:-1]

# Split the string numbers into multiple columns
df[['longMin', 'latMin', 'longMax', 'latMax']] = df['Bbox'].str.split(',', expand=True)

# Convert the new columns to numeric
df[['longMin', 'latMin', 'longMax', 'latMax']] = df[['longMin', 'latMin', 'longMax', 'latMax']].apply(pd.to_numeric)

# Drop the original column
df.drop(columns=['Bbox'], inplace=True)


listaDeListas = df.values.tolist()

FileNotFoundError: [Errno 2] No such file or directory: 'dbCoordenadas.xlsx'

In [ ]:
for coordenadas in listaDeListas:

    resolution = 10
    aoi_bbox = BBox(bbox=coordenadas, crs=CRS.WGS84)
    aoi_size = bbox_to_dimensions(aoi_bbox, resolution=resolution)
    
    print(f'Image shape at {coordenadas} m resolution: {aoi_size} pixels')
    if aoi_size[0] > 2500 or aoi_size[1] > 2500:
        print(f'Image shape at {coordenadas} m resolution: {aoi_size} pixels')

# Scripts

Estos scripts se usan para descirle a la API lo que tiene que descargar.

En este caso, son Todas las bandas del satelite.

In [ ]:
# Script todo

evalscriptAllBands = """
//VERSION=3
function setup() {
  return {
    input: [
      {
        bands: [
          "B01",
          "B02",
          "B03",
          "B04",
          "B05",
          "B06",
          "B07",
          "B08",
          "B8A",
          "B09",
          "B11",
          "B12",
        ],
        units: "DN",
      },
    ],
    output: {
      id: "default",
      bands: 12,
      sampleType: SampleType.UINT16,
    },
  }
}

function evaluatePixel(sample) {
  return [
    sample.B01,
    sample.B02,
    sample.B03,
    sample.B04,
    sample.B05,
    sample.B06,
    sample.B07,
    sample.B08,
    sample.B8A,
    sample.B09,
    sample.B11,
    sample.B12,
  ]
}
"""

# Rango de Tiempo
Este bloque determina el rango de tiempo a descargar. Originalmente está en 2019, 1, 1 - 2024, 4,3
Se deben definir cuántas imágenes descargar en dicho tiempo, por ejemplo 50 imágenes entre el 2019 - 2024

In [ ]:
start = datetime.datetime(2019, 1, 1)
end = datetime.datetime(2024, 4, 3)
n_chunks = 50
tdelta = (end - start) / n_chunks
edges = [(start + i * tdelta).date().isoformat() for i in range(n_chunks+1)]
slots = [(edges[i], edges[i + 1]) for i in range(len(edges) - 1)]

print("Monthly time windows:\n")
for slot in slots:
    print(slot)

# Descargar
El primer bloque es para procesamiento de imágenes.
El factor es principalmente para el brillo de la imagen. Estaba originalmente en 2 / 1e4, que es un buen factor para el espectro visible.

In [ ]:
factor = 2 / 1e4
clipRange= (0, 1)

In [ ]:
for coordenadas in listaDeListas:

    resolution = 10
    aoi_bbox = BBox(bbox=coordenadas, crs=CRS.WGS84)
    aoi_size = bbox_to_dimensions(aoi_bbox, resolution=resolution)

    for slot in slots:
        request_all_bands = SentinelHubRequest(
                evalscript= evalscriptAllBands,
                input_data=[
                    SentinelHubRequest.input_data(
                            data_collection=DataCollection.SENTINEL2_L2A.define_from(
                                name="s2", service_url="https://sh.dataspace.copernicus.eu"
                            ),
                            time_interval=slot,
                            other_args={"dataFilter": {"mosaickingOrder": "leastCC"}}           )
                    ],
                responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
                bbox=aoi_bbox,
                size=aoi_size,
                config=config,
            )
        all_bands_img = request_all_bands.get_data()

        # Carpetas
        nombreCarpetas = f'{coordenadas}'

        if not os.path.exists(nombreCarpetas):
            os.makedirs(nombreCarpetas)
    
        # Bandas RAW sin procesar
    
        bandaB01 = (np.clip(all_bands_img[0][:,:,0]*factor,*clipRange))*255
        bandaB01 = bandaB01.astype(np.uint8)
        bandaB01 = Image.fromarray(bandaB01)
        bandaB01.save(f'{nombreCarpetas}/{slot} B01.png')
    
        bandaB02 = (np.clip(all_bands_img[0][:,:,1]*factor,*clipRange))*255
        bandaB02 = bandaB02.astype(np.uint8)
        bandaB02 = Image.fromarray(bandaB02)
        bandaB02.save(f'{nombreCarpetas}/{slot} B02.png')
    
        bandaB03 = (np.clip(all_bands_img[0][:,:,2]*factor,*clipRange))*255
        bandaB03 = bandaB03.astype(np.uint8)
        bandaB03 = Image.fromarray(bandaB03)
        bandaB03.save(f'{nombreCarpetas}/{slot} B03.png')
    
        bandaB04 = (np.clip(all_bands_img[0][:,:,3]*factor,*clipRange))*255
        bandaB04 = bandaB04.astype(np.uint8)
        bandaB04 = Image.fromarray(bandaB04)
        bandaB04.save(f'{nombreCarpetas}/{slot} B04.png')
    
        bandaB05 = (np.clip(all_bands_img[0][:,:,4]*factor,*clipRange))*255
        bandaB05 = bandaB05.astype(np.uint8)
        bandaB05 = Image.fromarray(bandaB05)
        bandaB05.save(f'{nombreCarpetas}/{slot} B05.png')
    
        bandaB06 = (np.clip(all_bands_img[0][:,:,5]*factor,*clipRange))*255
        bandaB06 = bandaB06.astype(np.uint8)
        bandaB06 = Image.fromarray(bandaB06)
        bandaB06.save(f'{nombreCarpetas}/{slot} B06.png')
    
        bandaB07 = (np.clip(all_bands_img[0][:,:,6]*factor,*clipRange))*255
        bandaB07 = bandaB07.astype(np.uint8)
        bandaB07 = Image.fromarray(bandaB07)
        bandaB07.save(f'{nombreCarpetas}/{slot} B07.png')
    
        bandaB08 = (np.clip(all_bands_img[0][:,:,7]*factor,*clipRange))*255
        bandaB08 = bandaB08.astype(np.uint8)
        bandaB08 = Image.fromarray(bandaB08)
        bandaB08.save(f'{nombreCarpetas}/{slot} B08.png')
    
        bandaB8A = (np.clip(all_bands_img[0][:,:,8]*factor,*clipRange))*255
        bandaB8A = bandaB8A.astype(np.uint8)
        bandaB8A = Image.fromarray(bandaB8A)
        bandaB8A.save(f'{nombreCarpetas}/{slot} B8A.png')
    
        bandaB09 = (np.clip(all_bands_img[0][:,:,9]*factor,*clipRange))*255
        bandaB09 = bandaB09.astype(np.uint8)
        bandaB09 = Image.fromarray(bandaB09)
        bandaB09.save(f'{nombreCarpetas}/{slot} B09.png')
    
        bandaB11 = (np.clip(all_bands_img[0][:,:,10]*factor,*clipRange))*255
        bandaB11 = bandaB11.astype(np.uint8)
        bandaB11 = Image.fromarray(bandaB11)
        bandaB11.save(f'{nombreCarpetas}/{slot} B11.png')
    
        bandaB12 = (np.clip(all_bands_img[0][:,:,11]*factor,*clipRange))*255
        bandaB12 = bandaB12.astype(np.uint8)
        bandaB12 = Image.fromarray(bandaB12)
        bandaB12.save(f'{nombreCarpetas}/{slot} B12.png')
    
    
        # Bandas Procesadas
    
        trueColor = (np.clip(all_bands_img[0][:,:,[3,2,1]]*factor,*clipRange))*255
        trueColor = trueColor.astype(np.uint8)
        trueColor = Image.fromarray(trueColor)
        trueColor.save(f'{nombreCarpetas}/{slot} - trueColor.png')
    
        #falseColor = (np.clip(all_bands_img[0][:,:,[7,3,2]]*factor,*clipRange))*255
        #falseColor = falseColor.astype(np.uint8)
        #falseColor = Image.fromarray(falseColor)
        #falseColor.save(f'{slot} falseColor.png')




In [ ]:
trueColor.save(f'{nombreCarpetas}/{slot} - trueColor.png')